# Demonstrate how to access Tick History using python with a service account

In [66]:
# This is the GoogleBigQuery client library to hide the REST API calls
# Use e.g. 'pip install --upgrade google-cloud-bigquery' to add
from google.cloud import bigquery
# os is required to set the environment variable to point ot the JSON keyfile for authentication
import os
import pandas as pd

In [4]:
# Point GOOGLE_APPLICATION_CREDENTIALS to the credentials key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'dbd-pscacc-sdlc-prod-e91cde0f4754.json'

# Set the name of the GCP project where the queries are executed
gcp_query_project=u'dbd-pscacc-sdlc-prod'

# Here is the SQL query: this instructs BigQuery to do the heavy lifting

In [247]:
# Define the SQL query
query_expression = """
#standardSQL
DECLARE start_time TIMESTAMP DEFAULT TIMESTAMP("2020-11-23 09:30:00.000000");
DECLARE end_time TIMESTAMP DEFAULT TIMESTAMP("2020-11-23 16:00:00.000000");

SELECT RIC as symbol,
       Type,
       -- aggregate, aggregate, aggregate to use the capabilities of the cloud service
       Date_Time as date,
       "NSQ" as exchange_id,
       Price as price,
       Volume as volume
       
-- this is the market data table (ASQ/BTQ/PCQ/NSQ/NMQ/NAQ/NYQ/PNK available)
FROM `dbd-sdlc-prod.NSQ_NORMALISED.NSQ_NORMALISED`

-- ALWAYS(!) use a Date_Time constraint, otherwise the entire table will be scanned (e.g. NYQ > 75 TerraByte)
WHERE Type = "Trade" AND RIC = "AAPL.O" AND (Date_Time BETWEEN start_time AND end_time)

-- don't filter too much: it's more efficent to use e.g. RIC IN (list) or pattern matching than a single instrument
-- AND RIC LIKE '%.K'

ORDER BY Date_Time
"""

In [252]:
# Initialize a BigQuery client
bq_client_object = bigquery.Client(project = gcp_query_project)

# Initialize a BigQuery job config object to set various parameters
job_config_object = bigquery.QueryJobConfig()

# Dry run just gives back sizes and does not run the job!
# Alway run it before executing the query to make sure the data scan is what you'd expect
job_config_object.dry_run = True

In [253]:
# Initialize a BiqQuery job onject and pass it the config parameters
# Run to see results here
query_job_object = bq_client_object.query(query_expression, job_config = job_config_object)

if job_config_object.dry_run:
    print('Dry run - query will process {:,d} MB'.format(int(round(query_job_object.total_bytes_processed/1024**2))))
else:
    # This executes the query and puts it in a result iterator
    query_result = query_job_object.result()
    #print(type(query_result))
    
    # iterate through the result
    for row in query_result:
      print(row.date, row.Type, row.exchange_id, row.symbol, row.price, row.volume)

Dry run - query will process 1,853 MB


In [ ]:
# Initialize a BiqQuery job onject and pass it the config parameters
# Run to download results into a .csv file
query_job_object = bq_client_object.query(query_expression, job_config = job_config_object)

if job_config_object.dry_run:
    print('Dry run - query will process {:,d} MB'.format(int(round(query_job_object.total_bytes_processed/1024**2))))
else:
    # This executes the query and puts it in a result iterator
    query_result = query_job_object.result()
    #print(type(query_result))
    
    # iterate through the result
    empty_list = []
    for row in query_result:
      empty_list.append([row.date, row.Type, row.exchange_id, row.symbol, row.bid, row.bid_size, row.ask, row.ask_size])
    df = pd.DataFrame(empty_list)
    df.to_csv(r'C:\Users\robin\Desktop\quote_file.csv')